# Capacidad Hospitalaria y Demanda Sanitaria en Europa: Análisis europeo y caso de estudio en España

## Importando librerias

In [29]:
import pandas as pd 
import numpy as np 
import requests
import data_utils as du

## Cargando CSV

In [30]:
df_hospital = pd.read_csv('bed_hospital.csv.gz')
df_hospitalarias_estancia = pd.read_csv('hospitalaria_estancia.tsv.gz')
df_gasto_sanitario = pd.read_csv('gasto_sanitario.tsv.gz')

## Conectando con la API 

In [31]:
url = "https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/56934"
response = requests.get(url)

In [32]:
response.reason

'OK'

## Limpieza Tablas

### Tabla API - Instituto Nacional de Estadística

In [33]:
#leyendo mi tabla 
data = response.json()
df_population = pd.DataFrame(data)
df_population.head()

,COD,Nombre,FK_Unidad,FK_Escala,Notas,Data
0,ECP320,Total Nacional. Todas las edades. Total. Pobla...,3,1,[{'texto': 'https://www.ine.es/dyngs/INEbase/e...,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."
1,ECP319,Total Nacional. Todas las edades. Hombres. Pob...,3,1,NaN,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."
2,ECP318,Total Nacional. Todas las edades. Mujeres. Pob...,3,1,NaN,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."
3,ECP317,Total Nacional. 0 años. Total. Población. Núme...,3,1,NaN,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."
4,ECP316,Total Nacional. 0 años. Hombres. Población. Nú...,3,1,NaN,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."


In [34]:
#como los datos que quiero estan en la columna data, debo acceder a esos valores
#aqui estoy seleccionando la primera fila, tengo total nacional, todas las edades, toda la poblacion 
texto = df_population.loc[0, 'Nombre'] 

#busca las filas donde la columna Nombre contiene el texto guardado en 'texto'.
mask = df_population['Nombre'].str.contains(
    texto, 
    na=False
)

total_row = df_population[mask].iloc[0] #aqui acesso a ese valor
df_pop_total = pd.DataFrame(total_row['Data']) # y aqui seleccionamos ese valor y lo convertimos en un DF
df_pop_total.head()

,Fecha,FK_TipoDato,FK_Periodo,Anyo,Valor,Secreto
0,1735686000000,1,19,2025,49128297.0,False
1,1727733600000,1,22,2024,48999880.0,False
2,1719784800000,1,21,2024,48821936.0,False
3,1711922400000,1,20,2024,48701130.0,False
4,1704063600000,1,19,2024,48619695.0,False


In [35]:
#Arreglo la columna fecha, porque los datos de año son trismetrales y quiero quedarme con el mas reciente de ese año
df_pop_total['Fecha'] = pd.to_datetime(df_pop_total['Fecha'], unit='ms')
df_pop_total = df_pop_total.sort_values("Fecha") #asi lo pasamos a formato fecha y nos quedamos con el mas resiente

In [36]:
# me quedo con los ultimos valores de cada año
df_pop_total = df_pop_total.groupby('Anyo').tail(1)

#### Renombrando columnas

In [37]:
#Renombrare las columnas con las que voy a trabajar que serian 
df_pop_total = df_pop_total.rename(columns={
    "Anyo" : "year",
    "Valor" : "population"
})

In [38]:
#Quedandome con los valores que quiero y haciendolo DF
df_pop_total = df_pop_total[['year', 'population']]

In [41]:
# me quedo con los datos desde el 2013, porque as otras tablas empiezan desde 2013 
#mas que todo la de beds por habitantes
df_pop_total = df_pop_total [df_pop_total['year'] >= 2013]

In [42]:
df_pop_total.shape

(13, 2)